You might need to install this on your system:

apt-get install python3-opencv git

In [1]:
import os
#"""
# !rm k -r
if not os.path.isdir('k'):
 !git clone -b development12 https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull
#"""
!cd k && pip install .

Cloning into 'k'...
remote: Enumerating objects: 1384, done.
remote: Counting objects: 100% (833/833), done.
remote: Compressing objects: 100% (536/536), done.
remote: Total 1384 (delta 612), reused 511 (delta 296), pack-reused 551
Receiving objects: 100% (1384/1384), 15.00 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (968/968), done.
Processing /content/k
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cai: filename=cai-0.1.4-py3-none-any.whl size=57143 sha256=bd62f8033bdbae5b98a7b9ae59f5946b10ac3843dd3757acd9efff08e4c27277
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzn5x8kg/wheels/c1/8a/57/56dbba25eff58e52e5365435

In [2]:
import cai.layers
import cai.datasets
import cai.models
import cai.mobilenet_v3
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
print("CPU cores:", multiprocessing.cpu_count())
import psutil
print('RAM:', (psutil.virtual_memory().total / 1e9),'GB')
print(tf.config.list_physical_devices('GPU'))
import PIL
import cv2
#print("Pillow version:", PIL.PILLOW_VERSION)
from sklearn.metrics import classification_report

Tensorflow version: 2.7.0
Keras version: 2.7.0
CPU cores: 8
RAM: 54.76702208 GB
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!nvidia-smi

Fri Nov  5 03:38:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0


In [ ]:
if not os.path.isdir('up'):
  !git clone https://github.com/joaopauloschuler/PlantDoc-Object-Detection-Dataset.git up

Cloning into 'up'...
remote: Enumerating objects: 5190, done.
remote: Total 5190 (delta 0), reused 0 (delta 0), pack-reused 5190
Receiving objects: 100% (5190/5190), 941.89 MiB | 29.33 MiB/s, done.
Resolving deltas: 100% (2328/2328), done.
Checking out files: 100% (5199/5199), done.


In [ ]:
up_data_dir = os.getcwd()+"/up/TRAIN"
up_test_dir = os.getcwd()+"/up/TEST"
data_dir = "cropped_train/"
test_dir = "cropped_test/"
all_dir = "cropped_all/"

In [3]:
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(
      model.call,
      input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])

  graph_info = profile(forward_pass.get_concrete_function().graph,
                          options=ProfileOptionBuilder.float_operation())

  # The //2 is necessary since `profile` counts multiply and accumulate
  # as two flops, here we report the total number of multiply accumulate ops
  flops = graph_info.total_float_ops // 2
  return flops

In [ ]:
def CropImages(orig_data_dir, dest_data_dir, csv_file, include_original=False, verbose=True, prefix='img_'):
  # Load the CSV file.
  with open (csv_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(',')
      a_lines.append(a_line)
    line_count = line_count + 1
  #a_lines = np.array(a_lines, dtype=object)
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+a_line[3]
    dest_file_name = dest_folder_name+'/'+prefix+str(line_count)+'.jpg'
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    print(line_count,':', a_line, dest_file_name)
    if last_image_file != orig_file_name:
      if os.path.isfile(orig_file_name):
        img = cv2.imread(orig_file_name)
        #cv2_imshow(img)
        last_image_file = orig_file_name
        can_load = True
        original_count = original_count + 1
        if verbose: print("Original imgage shape",img.shape)
        if (include_original):
          if not os.path.isdir(dest_folder_name):
            os.mkdir(dest_folder_name)
          cv2.imwrite(dest_folder_name+'/ori_'+prefix+str(original_count)+'.jpg',img)  
      else:
        if verbose: print('File does not exist:', orig_file_name)
        can_load = False
        #break
    if (can_load):
      y1 = int(a_line[4])
      x1 = int(a_line[5])
      y2 = int(a_line[6])
      x2 = int(a_line[7])

      x_min = np.min([x1, x2])
      x_max = np.max([x1, x2])
      y_min = np.min([y1, y2])
      y_max = np.max([y1, y2])

      crop_img = img[x_min:x_max, y_min:y_max]
      if verbose: print("Cropped imgage shape",crop_img.shape,':', x_min, x_max, y_min, y_max,' deltas:',x_max - x_min, y_max - y_min)
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if ( (crop_img.shape[0]==0) or (crop_img.shape[1]==0)):
        failed_count = failed_count + 1
        if verbose: print("Failed cropping.");
      else:
        cv2.imwrite(dest_file_name,crop_img)
    else:
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(data_dir):
  CropImages(orig_data_dir=up_data_dir, dest_data_dir=data_dir, csv_file='up/train_labels.csv', include_original=False, prefix='train_')

Streaming output truncated to the last 5000 lines.
6274 : ['3_21_14-Caption-Black-Rot-leaf-infection.-265x265.jpg', '265', '265', 'grape leaf black rot', '1', '9', '136', '239'] cropped_train//grape leaf black rot/train_6274.jpg
Original imgage shape (265, 265, 3)
Cropped imgage shape (230, 135, 3) : 9 239 1 136  deltas: 230 135
6275 : ['603-116-0.jpg', '502', '670', 'Tomato mold leaf', '1', '158', '358', '670'] cropped_train//Tomato mold leaf/train_6275.jpg
Original imgage shape (670, 502, 3)
Cropped imgage shape (512, 357, 3) : 158 670 1 358  deltas: 512 357
6276 : ['IMG_4280.JPG.jpg', '1600', '1200', 'Cherry leaf', '117', '194', '655', '589'] cropped_train//Cherry leaf/train_6276.jpg
Original imgage shape (1200, 1600, 3)
Cropped imgage shape (395, 538, 3) : 194 589 117 655  deltas: 395 538
6277 : ['IMG_4280.JPG.jpg', '1600', '1200', 'Cherry leaf', '803', '264', '1018', '658'] cropped_train//Cherry leaf/train_6277.jpg
Cropped imgage shape (394, 215, 3) : 264 658 803 1018  deltas: 394

In [ ]:
if not os.path.isdir(test_dir):
  CropImages(orig_data_dir=up_test_dir, dest_data_dir=test_dir, csv_file='up/test_labels.csv', include_original=False, prefix='test_')

0 : ['Black%20rot%20on%20foliage.jpg', '1060', '795', 'grape leaf black rot', '1', '58', '859', '732'] cropped_test//grape leaf black rot/test_0.jpg
Original imgage shape (795, 1060, 3)
Cropped imgage shape (674, 858, 3) : 58 732 1 859  deltas: 674 858
1 : ['pepper_leaf.jpg', '300', '400', 'Bell_pepper leaf', '40', '58', '251', '392'] cropped_test//Bell_pepper leaf/test_1.jpg
Original imgage shape (400, 300, 3)
Cropped imgage shape (334, 211, 3) : 58 392 40 251  deltas: 334 211
2 : ['depositphotos_1323264-Raspberry-leaf-on-white.jpg', '1024', '772', 'Raspberry leaf', '98', '75', '964', '710'] cropped_test//Raspberry leaf/test_2.jpg
Original imgage shape (772, 1024, 3)
Cropped imgage shape (635, 866, 3) : 75 710 98 964  deltas: 635 866
3 : ['dscn6689.jpg', '1024', '768', 'Tomato mold leaf', '140', '102', '887', '712'] cropped_test//Tomato mold leaf/test_3.jpg
Original imgage shape (768, 1024, 3)
Cropped imgage shape (610, 747, 3) : 102 712 140 887  deltas: 610 747
4 : ['IMG_1246.jpg', '

In [ ]:
print(os.listdir(data_dir))
print(os.listdir(test_dir))
if not os.path.isdir('cropped_test/Potato leaf'):
  !mkdir 'cropped_test/Potato leaf'
if not os.path.isdir('cropped_test/Tomato two spotted spider mites leaf'):
  !mkdir 'cropped_test/Tomato two spotted spider mites leaf'

['Raspberry leaf', 'Blueberry leaf', 'Cherry leaf', 'Corn leaf blight', 'Apple rust leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Potato leaf', 'Peach leaf', 'Tomato leaf mosaic virus', 'Tomato leaf', 'Tomato Septoria leaf spot', 'Tomato leaf bacterial spot', 'Bell_pepper leaf', 'Corn rust leaf', 'Apple leaf', 'grape leaf', 'Potato leaf late blight', 'Corn Gray leaf spot', 'Tomato leaf late blight', 'Apple Scab Leaf', 'Tomato leaf yellow virus', 'Tomato Early blight leaf', 'Bell_pepper leaf spot', 'grape leaf black rot', 'Tomato two spotted spider mites leaf', 'Strawberry leaf', 'Tomato mold leaf', 'Potato leaf early blight']
['Raspberry leaf', 'Blueberry leaf', 'Cherry leaf', 'Corn leaf blight', 'Apple rust leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Peach leaf', 'Tomato leaf mosaic virus', 'Tomato leaf', 'Tomato Septoria leaf spot', 'Tomato leaf bacterial spot', 'Bell_pepper leaf', 'Corn rust leaf', 'Apple leaf', 'grape leaf', 'Potato leaf late blight', 'Corn Gray

In [ ]:
if not os.path.isdir(all_dir):
  !mkdir cropped_all
  ! cp -r cropped_test/* cropped_all/
  ! cp -r cropped_train/* cropped_all/

In [4]:
datagen = cai.util.create_image_generator(rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, channel_shift_range=0.1) 

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

In [5]:
num_classes = 29
batch_size = 64
epochs = 75
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=all_dir, lab=False, 
  verbose=True, bipolar=False, base_model_name='plant_doc',
  has_training=True, has_validation=True, has_testing=True,
  smart_resize=True,
  training_size=0.65, validation_size=0.15, test_size=0.2,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  29  classes.
smart resize is enabled.
loading train images
train shape is: (5761, 224, 224, 3)
loading validation images
validation shape is: (1333, 224, 224, 3)
loading test images
test shape is: (1789, 224, 224, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0
Loaded.
(5761, 224, 224, 3) (1333, 224, 224, 3) (1789, 224, 224, 3)
(5761, 29) (1333, 29) (1789, 29)


In [6]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 2, 13]:
      basefilename = 'JP36A01-MovileNetV3Large-CroppedPlantDoc-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.mobilenet_v3.MobileNetV3Large(
          input_shape=input_shape,
          alpha=1.0,
          minimalistic=False,
          include_top=True,
          input_tensor=None,
          classes=num_classes,
          pooling=None,
          dropout_rate=0.2)
      else:
        model = cai.mobilenet_v3.kMobileNetV3Large(
          input_shape=input_shape,
          alpha=1.0,
          minimalistic=False,
          include_top=True,
          input_tensor=None,
          classes=num_classes,
          pooling=None,
          dropout_rate=0.2,
          kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)

      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): 
        model.summary()
        print('model flops:',get_flops(model))

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=max([multiprocessing.cpu_count(), 4]) # this option 
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [9]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False)

Running: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           ['input_2[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['Conv_pad[0][0]']               
                                )                                                                 
                              

# Fitting

In [ ]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=True)

Running: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
Epoch 1/75
91/91 [==============================] - 47s 202ms/step - loss: 3.2508 - accuracy: 0.1139 - val_loss: 3.2479 - val_accuracy: 0.0953

Epoch 00001: val_accuracy improved from -inf to 0.09527, saving model to JP36A01-MovileNetV3Large-CroppedPlantDoc--1-best_result.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/75
91/91 [==============================] - 17s 179ms/step - loss: 2.9769 - accuracy: 0.1528 - val_loss: 3.2650 - val_accuracy: 0.0953

Epoch 00002: val_accuracy did not improve from 0.09527
Epoch 3/75
91/91 [==============================] - 19s 176ms/step - loss: 2.9081 - accuracy: 0.1727 - val_loss: 3.3753 - val_accuracy: 0.0915

Epoch 00003: val_accuracy did not improve from 0.09527
Epoch 4/75
91/91 [==============================] - 17s 178ms/step - loss: 2.8865 - accuracy: 0.1774 - val_loss: 3.6844 - val_accuracy: 0.0915

Epoch 00004: val_accuracy did not improve from 0.09527
Epoch 5/75
91/91 [==============================] - 19s 178ms/step - loss: 2.8436 - accuracy: 0.1771 - val_loss: 3.6248 - val_accuracy: 0.0555

Epoch 00005: val_accuracy did not improve from 0.09527
Epoch 6/75
91/91 [==============================] - 17s 176ms/step - loss: 2.8142 - accuracy: 0.1930 - val_loss: 4.1670 - val_accuracy: 0.0623

Epoch 00006: val_accuracy did not improve from 0.09527
Epoch

# Test Results

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True)

Running: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
Best Model Results: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
56/56 [==============================] - 2s 14ms/step - loss: 1.9472 - accuracy: 0.4645
loss 1.94724702835083
acc 0.46450531482696533
Finished: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
Running: JP36A01-MovileNetV3Large-CroppedPlantDoc-2
Best Model Results: JP36A01-MovileNetV3Large-CroppedPlantDoc-2
56/56 [==============================] - 8s 41ms/step - loss: 1.4877 - accuracy: 0.6165
loss 1.487656831741333
acc 0.616545557975769
Finished: JP36A01-MovileNetV3Large-CroppedPlantDoc-2
Running: JP36A01-MovileNetV3Large-CroppedPlantDoc-13
Best Model Results: JP36A01-MovileNetV3Large-CroppedPlantDoc-13
56/56 [==============================] - 13s 48ms/step - loss: 1.5645 - accuracy: 0.6534
loss 1.5645225048065186
acc 0.6534376740455627
Finished: JP36A01-MovileNetV3Large-CroppedPlantDoc-13


# Calculate F1

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True, calc_f1=True)

Running: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
Best Model Results: JP36A01-MovileNetV3Large-CroppedPlantDoc--1
56/56 [==============================] - 2s 14ms/step - loss: 1.9472 - accuracy: 0.4645
loss 1.94724702835083
acc 0.46450531482696533
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.4444    0.1143    0.1818        35
           1     0.5556    0.2000    0.2941        50
           2     0.1847    0.8056    0.3005        36
           3     0.6140    0.5385    0.5738        65
           4     0.2330    0.4528    0.3077        53
           5     0.6641    0.5030    0.5724       169
           6     0.2157    0.2292    0.2222        48
           7     0.4118    0.4375    0.4242        16
           8     0.8077    0.5676    0.6667        74
           9     0.5000    0.2692    0.3500        26
          10     0.6391    0.6855    0.6615       124
          11    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best Model Results: JP36A01-MovileNetV3Large-CroppedPlantDoc-2
56/56 [==============================] - 9s 41ms/step - loss: 1.4877 - accuracy: 0.6165
loss 1.487656831741333
acc 0.616545557975769
Predicted Shape: (1789, 29)
Pred classes shape: (1789,)
Test classes shape: (1789,)
              precision    recall  f1-score   support

           0     0.5000    0.3429    0.4068        35
           1     0.5088    0.5800    0.5421        50
           2     0.5106    0.6667    0.5783        36
           3     0.6076    0.7385    0.6667        65
           4     0.3846    0.4717    0.4237        53
           5     0.7041    0.7041    0.7041       169
           6     0.5517    0.3333    0.4156        48
           7     0.4667    0.4375    0.4516        16
           8     0.8169    0.7838    0.8000        74
           9     0.7895    0.5769    0.6667        26
          10     0.8081    0.6452    0.7175       124
          11     0.0000    0.0000    0.0000         3
          12     